<h1><center>Простые векторные модели текста</center></h1>

<img src="pipeline_vec.png" alt="pipeline.png" style="width: 400px;"/>

### Задача: классификация твитов по тональности

В этом занятии мы познакомимся с распространенной задачей в анализе текстов: с классификацией текстов на классы.

В рассмотренном тут примере классов будет два: положительный и отрицательный, такую постановку этой задачи обычно называют классификацией по тональности или sentiment analysis.

Классификацию по тональности используют, например, в рекомендательных системах и при анализе отзывов клиентов, чтобы понять, понравилось ли людям кафе, кино, etc.

Более подробно мы рассмотрим данную задачу и познакомимся с более сложными методами её решения в семинаре 3, а здесь разберем простые подходы, основанные на методе мешка слов.

У нас есть [данные постов в твиттере](http://study.mokoron.com/), про из которых каждый указано, как он эмоционально окрашен: положительно или отрицательно. 

**Задача**: построить модель, которая по тексту поста предсказывает его эмоциональную окраску.


Скачиваем данные: [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [5]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-09-12 16:38:20--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-09-12 16:38:21--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5a7eee673483b937e76fb66523.dl.dropboxusercontent.com/cd/0/inline/Bsy5KzJN9lQQ0XA5rrdeBDNyNeubQa7hzNvTXtCD5IFEK_4suXN6f8R-_T-TcqJId3gQn1604i1JENpO6Wo45p2_RzuFXFaiL_hKMuXMeXe8MxkfJcb0__hzpkqav94l544z8zR30irb8cBE63c58UCh-acR2MNdM0Wx7s6s1_RIzw/file# [following]
--2022-09-12 16:38:22--  https://uc5a7eee673483b937e76fb66523.dl.dropboxusercontent.com/cd/0/inline/Bsy5KzJN9lQQ0XA5rrdeBDNyNeubQa7hzNvTXtCD5IFEK_4suXN6f8R-_T-TcqJId3gQn1604i1JENpO6Wo45p2_RzuFXFaiL_hKMuXMeX

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [9]:
positive = pd.read_csv('./positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('./negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [10]:
df.sample(5)

,text,label
71243,"То чувство, когда у тебя не работает ВК и ты никак не можешь поздравить остальных своих друзей ;(\n#НГ #пздц",negative
31533,По ТНТ идёт полицейская академия. Ностальгия :(,negative
81528,"Мой телефон реагирует на отмерзшие пальцы в перчатках, правда писать не очень удобно:|",negative
109614,@_Angel_OF_Lord А ПОЧЕМУ ВОКРУГ СТОЛА?\nПРОСТО МОЯ БОЛЬНАЯ ФАНТАЗИЯ УЖЕ СТРОИТ НЕ ОЧЕНЬ ПРИЛИЧНЫЕ КАРТИНКИ В УМЕ:DD #жаркийчетверг,positive
109132,"-Вы расстались, из-за чего? \n-Да если бы я сам это знал(",negative


Воспользуемся функцией для предобработки текста, которую мы написали в прошлом семинаре:

In [11]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [12]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [13]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

100%|██████████| 226834/226834 [00:54<00:00, 4168.88it/s]


,text,label,lemmas
75734,мне мало мало мало мне малооо зарядки на плеере:(,negative,мало мало мало малооо зарядка плеер
22310,@ehnevermind Я никогда его не найду &gt;;(,negative,ehnevermind найти
73725,"RT @igajaqytmu: смотрит на вас, как на вебкамеру. И странно скалится при этом %))",positive,igajaqytmu смотреть вебкамера странно скалиться
16467,всё вернулось на свои места))\nопять в чс\nя счастлив\nтварь,positive,вернуться свой место счастливый тварь
9758,@funkyboyakeem я не пила мандариновую водку) и с возрастом не угадал,positive,funkyboyakeem пила мандариновый водка возраст угадать


Разбиваем на train и test:

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df.lemmas, df.label)

## Мешок слов (Bag of Words, BoW)


In [44]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

... Но сперва пару слов об n-граммах. Что такое n-граммы:

In [45]:
from nltk import ngrams

In [46]:
sent = 'Факультет компьютерных наук Высшей школы экономики'.split()
list(ngrams(sent, 1)) # униграммы

[('Факультет',),
 ('компьютерных',),
 ('наук',),
 ('Высшей',),
 ('школы',),
 ('экономики',)]

In [19]:
list(ngrams(sent, 2)) # биграммы

[('Факультет', 'компьютерных'),
 ('компьютерных', 'наук'),
 ('наук', 'Высшей'),
 ('Высшей', 'школы'),
 ('школы', 'экономики')]

In [20]:
list(ngrams(sent, 3)) # триграммы

[('Факультет', 'компьютерных', 'наук'),
 ('компьютерных', 'наук', 'Высшей'),
 ('наук', 'Высшей', 'школы'),
 ('Высшей', 'школы', 'экономики')]

In [21]:
list(ngrams(sent, 5)) # ... пентаграммы?

[('Факультет', 'компьютерных', 'наук', 'Высшей', 'школы'),
 ('компьютерных', 'наук', 'Высшей', 'школы', 'экономики')]

Итак, мы хотим преобразовать наши обработанные данные в вектора с помощью мешка слов. Мешок слов можно строить как для отдельных слов (лемм в нашем случае), так и для n-грамм, и это может улучшать качество. 

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [47]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

ngram_range отвечает за то, какие n-граммы мы используем в качестве признаков:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В vec.vocabulary_ лежит словарь: соответствие слов и их индексов в словаре:

In [23]:
list(vec.vocabulary_.items())[:10]

[('жаль', 110128),
 ('стать', 153946),
 ('редко', 147080),
 ('заходить', 112987),
 ('новогодний', 130623),
 ('каникулы', 116354),
 ('пролететь', 144046),
 ('день', 106675),
 ('заметить', 112070),
 ('успеть', 160401)]

In [24]:
bow[0]

<1x169048 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

Теперь у нас есть вектора, на которых можно обучать модели! 

In [25]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Посмотрим на качество классификации на тестовой выборке. Для этого выведем classification_report из модуля [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

В качестве целевой метрики качества будем рассматривать macro average f1-score.

In [26]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.74      0.73      0.74     28385
    positive       0.73      0.75      0.74     28324

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Попробуем сделать то же самое для триграмм:

In [27]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.97      0.53      0.68     51252
    positive       0.16      0.84      0.27      5457

    accuracy                           0.56     56709
   macro avg       0.56      0.69      0.48     56709
weighted avg       0.89      0.56      0.64     56709



Видим, что качество существенно хуже. Ниже мы поймем, почему это так.

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TFIDF(t,d,D) = TF(t,d) \times IDF(i, D)$$

Ключевая идея этого подхода – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.70      0.74      0.72     26327
    positive       0.77      0.72      0.74     30382

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.74      0.73      0.73     56709



В этот раз получилось хуже, чем с помощью простого CountVectorizer, то есть использование tf-idf не дало улучшений в качестве. 

## О важности эксплоративного анализа

Иногда в ходе стандартного препроцессинга теряются важные признаки. Посмотрим, что будет если не убирать пунктуацию?

In [30]:
df.sample()

,text,label,lemmas
52766,"RT @tataholiday: хочу вернуть старые времена, опять :( \n#miss",negative,tataholiday хотеть вернуть старое время miss


In [31]:
df['new_lemmas'] = df.text.apply(lambda x: x.lower())
df.sample(3)

,text,label,lemmas,new_lemmas
24694,Q: подпишись\nпожалуйста*) A: http://t.co/XKTaDVlUP2,positive,подписаться пожалуйста http xktadvlup,q: подпишись\nпожалуйста*) a: http://t.co/xktadvlup2
54857,"В свои 11 лет, я встречалась с 18-и летним парнем. Мне так жалко таких парней, их друзья оказывается так издеваются. Мы встречались 2 года((",negative,свой встречаться летний парень жалко парный друг оказываться издеваться встречаться,"в свои 11 лет, я встречалась с 18-и летним парнем. мне так жалко таких парней, их друзья оказывается так издеваются. мы встречались 2 года(("
7777,"Общажный движ закончился тем что все спят, а я нихуя:)",positive,общажный движ закончиться весь спать нихуй,"общажный движ закончился тем что все спят, а я нихуя:)"


In [32]:
x_train, x_test, y_train, y_test = train_test_split(df.new_lemmas, df.label)

In [33]:
from nltk import word_tokenize

vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27863
    positive       1.00      1.00      1.00     28846

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Как можно видеть, если оставить пунктуацию, то все метрики равны 1. 

In [34]:
len(vec.vocabulary_), len(clf.coef_[0])

(266706, 266706)

In [35]:
importances = list(zip(vec.vocabulary_, clf.coef_[0]))
importances[0]

('@', 0.1471878855818156)

In [36]:
sorted_importances = sorted(importances, key = lambda x: -x[1])
sorted_importances[:10]

[('что', 58.77436936506602),
 ('//t.co/cuphpysvcp', 27.295181101473574),
 ('у', 12.578240551762557),
 ('daromand77', 10.80497075752631),
 ('кюхенио', 9.111201609919437),
 ('учете', 8.102007005355661),
 ('50', 7.678739507821211),
 ('него', 5.254932152879905),
 ('коллега', 4.744012822521124),
 ('останавливало', 4.679962041879572)]

Посмотрим, как один из наиболее значительных токенов справится с классификацией безо всякого машинного обучения:

In [37]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32901
    positive       0.83      1.00      0.90     23808

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



Можно видеть, что это уже позволяет достаточно хорошо классифицировать тексты.

## Символьные n-граммы

Теперь в качестве признаком используем, например, униграммы символов:

In [38]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/Users/aksenov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.99      1.00      0.99     27773
    positive       1.00      0.99      1.00     28936

    accuracy                           0.99     56709
   macro avg       0.99      0.99      0.99     56709
weighted avg       0.99      0.99      0.99     56709



Таким образом, становится понятно, почему на этих данных качество классификации 1. Так или иначе, на символах классифицировать тоже можно.

Ещё одна замечательная особенность символьных признаков: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Итоги

 На этом занятии мы
* познакомились с задачей бинарной классификации текстов.

* научились строить простые признаки на основе метода "мешка слов" с помощью библиотеки sklearn: CountVectorizer и TfidfVectorizer.

* использовали для классификации линейную модель логистической регрессии.

* поняли, что многое зависит от подхода к предобработки текста и от признаков, которые используются в модели.

* увидели, что в некоторых задачах важно использование каждого символа из текста, в том числе пунктуации.

На следующих занятиях мы рассмотрим более сложные модели построения признаков и классификации текстов.